### [Boids by Craig Reynolds](https://www.red3d.com/cwr/boids/)

| Separation | Alignment | Cohesion |
| --- | --- | --- |
| Steer to avoid crowding local flockmates | Steer towards the average heading of local flockmates | Steer to move toward the average position of local flockmates |
| ![separation](https://www.red3d.com/cwr/boids/images/separation.gif) | ![alignment](https://www.red3d.com/cwr/boids/images/alignment.gif) | ![cohesion](https://www.red3d.com/cwr/boids/images/cohesion.gif) |

![neighbourhood](https://www.red3d.com/cwr/boids/images/neighborhood.gif)

Neighbourhood

---

Example: [The Boids! - UCL](https://rits.github-pages.ucl.ac.uk/doctoral-programming-intro/02-novice/100Boids.html)

In [ ]:
%matplotlib widget
# Dependencies
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np

# Settings
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
w_ = 6
h_ = 3

In [ ]:
class Boid(object):
    def __init__(self, pos, vel, ax):
        self.pos = pos
        self.vel = vel
        self.max_vel = 2
        unit_vel = self.vel / np.linalg.norm(self.vel)
        self.body = ax.quiver(self.pos[0], self.pos[1], unit_vel[0], unit_vel[1], scale=50, angles='xy', color=(1,1,1), pivot='middle')
        self.force = np.zeros(2)

    def separation(self, boids):
        separation = 1 * np.mean(self.pos - np.array([boid.pos for boid in boids]), axis=0)
        return separation

    def alignment(self, boids):
        alignment = 1 * np.mean([boid.vel/np.linalg.norm(boid.vel) for boid in boids], axis=0)
        return alignment
    
    def cohesion(self, boids):
        cohesion = 1 * (np.mean([boid.pos for boid in boids], axis=0) - self.pos)
        return cohesion

    def flock(self, boids):
        self.force = np.zeros(2)
        max_dist = np.float32(0.5)
        neighbourhood = np.extract([np.linalg.norm(self.pos - boid.pos) for boid in boids] <= max_dist, boids)
        self.force += self.separation(neighbourhood)
        self.force += self.alignment(neighbourhood)
        self.force += self.cohesion(neighbourhood)

    def update(self, dti):
        self.vel += dti * self.force
        self.vel = self.max_vel * (self.vel/np.linalg.norm(self.vel)) if np.linalg.norm(self.vel) > self.max_vel else self.vel
        self.pos += dti * self.vel
        self.pos = np.mod(self.pos, [w_, h_])
        self.body.set_offsets(self.pos)
        unit_vel = self.vel / np.linalg.norm(self.vel)
        self.body.set_UVC(unit_vel[0], unit_vel[1])

In [ ]:
try:
    ani.event_source.stop()
except (NameError, AttributeError) as e:
    pass

# Plot setup
fig = plt.figure(figsize=(6,3))
fig.set_label("Boids")
ax = fig.subplots()
ax.set_facecolor((0,0,0))
ax.set_xlim(0, w_)
ax.set_ylim(0, h_)
ax.set_xticks([])
ax.set_yticks([])

# Boids
boids = [Boid(np.array([w_/2, h_/2]), 4*(np.random.rand(2)-0.5), ax) for i in range(50)]

# Time period
t_f = 30
t = np.linspace(0, t_f, num=t_f*30)
dt = np.diff(t)
dt = np.append(dt, [dt[-1]])

def update(dt_i):
    for boid in boids:
        boid.flock(boids)
        boid.update(dt_i)

ani = matplotlib.animation.FuncAnimation(fig, update, frames=dt, interval=1000*np.mean(dt), repeat=False)
fig.tight_layout()
plt.show()
